In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
from SAC.SACAgent import SAC
from SAC_Distillation.Trajectories import SAC_ExperienceBuffer
from Hyperparameters import HYPERPARAMS as params
import numpy as np
import torch
import wandb

c:\Users\Fede\anaconda3\envs\mlagents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(project="SAC", entity="fede-")
wandb.config.update(params['sac_distilled'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.config.update({"device": device})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rullofederico16 (fede-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
def relocate_agents(env):
    return list(env.agents)  # simplified

# New helper to extract observation data for an agent
def get_agent_obs(obs, agent):
    agent_data = obs[agent]
    return np.array(agent_data[1]), np.array(agent_data[2])

In [4]:
env = UE(file_name="Env/DroneFlightv1", seed=1, no_graphics_monitor=True)
env = UPZBE(env)
env.reset()

{'Drone?team=0?agent_id=3': [array([[[0.20784314, 0.21568628, 0.21960784, ..., 0.3647059 ,
           0.36862746, 0.36862746],
          [0.20784314, 0.21568628, 0.21960784, ..., 0.36862746,
           0.36862746, 0.36862746],
          [0.20784314, 0.21176471, 0.21568628, ..., 0.37254903,
           0.37254903, 0.37254903],
          ...,
          [0.21960784, 0.21568628, 0.21960784, ..., 0.39607844,
           0.80784315, 0.80784315],
          [0.21960784, 0.21568628, 0.21960784, ..., 0.39215687,
           0.8039216 , 0.8039216 ],
          [0.21568628, 0.21568628, 0.21960784, ..., 0.3882353 ,
           0.8039216 , 0.8039216 ]],
  
         [[0.24705882, 0.2509804 , 0.25490198, ..., 0.45882353,
           0.45882353, 0.45882353],
          [0.24705882, 0.2509804 , 0.25490198, ..., 0.45882353,
           0.4627451 , 0.4627451 ],
          [0.24313726, 0.2509804 , 0.2509804 , ..., 0.4627451 ,
           0.46666667, 0.46666667],
          ...,
          [0.23137255, 0.23137255, 0.23

In [5]:
agents = relocate_agents(env)
print(agents)

['Drone?team=0?agent_id=0', 'Drone?team=0?agent_id=1', 'Drone?team=0?agent_id=10', 'Drone?team=0?agent_id=11', 'Drone?team=0?agent_id=2', 'Drone?team=0?agent_id=3', 'Drone?team=0?agent_id=4', 'Drone?team=0?agent_id=5', 'Drone?team=0?agent_id=6', 'Drone?team=0?agent_id=7', 'Drone?team=0?agent_id=8', 'Drone?team=0?agent_id=9']


In [6]:
Buffer = SAC_ExperienceBuffer(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape,env.action_space(agents[0]).shape, params['sac_distilled'])

In [7]:
brain = SAC(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape[0],len(agents), params['sac_distilled'])

In [8]:
# for s in range(1, params['sac_distilled'].seed_episodes):
#     obs, done, t = env.reset(), [False for _ in env.agents], 0
#     while not all(done) or t < params['sac_distilled'].n_steps_random_exploration:
#         actions = {}
#         log_probs = {}
#         values = {}
#         agents = relocate_agents(env)
#         for agent in agents:
#             if agent not in obs.keys():
#                 continue
#             actions[agent] = env.action_space(agent).sample()
#             obs1, obs2 = get_agent_obs(obs, agent)
#             v1,v2 = brain.get_values(obs1,obs2, actions[agent],t)
#             values[agent] = torch.min(v1,v2)

#         actions = {agent: act for agent, act in actions.items() if agent in set(env.agents)}
#         next_obs, reward, done, _ = env.step(actions)

#         for agent in agents:
#             if agent not in next_obs.keys():
#                 continue
#             next_obs1, next_obs2 = get_agent_obs(next_obs, agent)
#             Buffer.store(obs1, obs2, actions[agent], reward[agent], next_obs1, next_obs2, done[agent])
#         obs = next_obs
#         done = [done[agent] for agent in agents if agent in done.keys()]
#         t += 1
#     print(f'Finished episode {s}')

# # Buffer.compute_advantages()
# print("Finished Rnd Exploration")
# env.close()

In [9]:
# Buffer.save('Collected_Experience/init_experience.npz')

In [10]:
Buffer.load('Collected_Experience/init_experience.npz')
Buffer.size=min(len(Buffer.camera_obs)+1, Buffer.buffer_size)
Buffer.ptr=Buffer.size-1 % Buffer.buffer_size
print(len(Buffer))
print(Buffer.ptr)
print(Buffer.size)

300001
300000
300001


In [11]:
brain.train(Buffer,step = params['sac_distilled'].seed_episodes*params['sac_distilled'].n_steps_random_exploration)

RuntimeError: Expected canUse32BitIndexMath(input) && canUse32BitIndexMath(output) to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [ ]:
brain.save("SavedModels/SAC_checkpoint.pth")
print("Checkpoint saved successfully.")
env.close()

In [ ]:
# brain.load("SavedModels/SAC_distilled_checkpoint.pth")
# print("Checkpoint loaded successfully.")

Checkpoint loaded successfully.


In [ ]:
env = UE(file_name="Env/DroneFlightv1", seed=1, no_graphics_monitor=True)
env = UPZBE(env)
env.reset()
agents = relocate_agents(env)

In [ ]:
steps=0
obs = env.reset()
mean_reward = -np.inf  # or 0
max_steps = params['sac_distilled'].max_steps
n_steps = params['sac_distilled'].n_steps

In [ ]:
while steps < max_steps:
    episode_rewards = []
    t = 0
    done_flags = {agent: False for agent in env.agents}
    while not all(done_flags.values()) and t < n_steps:
        actions, values = {}, {}
        agents = relocate_agents(env)

        for agent in env.agents:
            if agent not in obs:
                continue


            camera_obs, pos_obs = get_agent_obs(obs, agent)
            
            action_tensor, _ = brain.get_action(camera_obs, pos_obs, train=True)
            v1, v2 = brain.get_values(camera_obs, pos_obs, action_tensor, steps + t)
            value = torch.min(v1, v2)


            actions[agent] = action_tensor.cpu().detach().numpy().flatten().tolist()
            values[agent] = value

        # Step environment safely with active agent actions
        next_obs, rewards, dones, _ = env.step(actions)

        for agent in actions.keys():
            if agent not in next_obs:
                continue

            camera_obs, pos_obs = get_agent_obs(obs, agent)
            next_camera_obs, next_pos_obs = get_agent_obs(next_obs, agent)
            reward = rewards[agent] * 10  # scale reward appropriately
            done = dones[agent] 

            Buffer.store(
                camera_obs,
                pos_obs,
                actions[agent],
                reward,
                next_camera_obs,
                next_pos_obs,
                done
            )

            episode_rewards.append(reward)
            done_flags[agent] = done

        obs = next_obs
        t += 1

        # Remove terminated agents explicitly
        terminated_agents = [agent for agent, d in done_flags.items() if d]
        for agent in terminated_agents:
            del done_flags[agent]

    # Compute reward properly
    current_reward = np.mean(episode_rewards)

    # Checkpoint the model if improved
    if current_reward > mean_reward:
        brain.save("SavedModels/SAC_distilled_trained.pth")
        print(f"Checkpoint saved successfully at step {steps} with reward {current_reward:.3f}.")
        mean_reward = current_reward

    steps += t

    # Perform training step
    actor_loss, critic_loss, entropy_loss = brain.train(Buffer, steps)

    # Log meaningful metrics clearly
    mean_val = np.mean([value.mean().item() for value in values.values() if value is not None])
    
    # Safely handle extreme critic losses
    if critic_loss > 1e6:
        brain.load("SavedModels/SAC_distilled_trained.pth")
        print("Critical critic loss encountered. Restored previous checkpoint.")

    wandb.log({
        "Mean Reward": current_reward,
        "Steps": steps,
        "Actor Loss": actor_loss,
        "Critic Loss": critic_loss,
        "Entropy Loss": entropy_loss,
        "Mean Value": mean_val
    })

env.close()

Checkpoint saved successfully at step 0 with reward -3.683.


c:\Users\Fede\anaconda3\envs\mlagents\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Fede\anaconda3\envs\mlagents\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Critical critic loss encountered. Restored previous checkpoint.
Critical critic loss encountered. Restored previous checkpoint.
Critical critic loss encountered. Restored previous checkpoint.
Critical critic loss encountered. Restored previous checkpoint.
Critical critic loss encountered. Restored previous checkpoint.
Critical critic loss encountered. Restored previous checkpoint.


KeyboardInterrupt: 